In [22]:
import warnings
warnings.filterwarnings('ignore')

# tools
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# data processing/sampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# model/metrics
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [23]:
# Load the data
file_path = Path('C:/Users/panka/Documents/Homework/Module_11_Challenge/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

In [24]:
# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [25]:
# Create our features
X = df.drop("loan_status", axis=1)
X = pd.get_dummies(X)

# Create our target
y = df["loan_status"]

In [26]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [27]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [28]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(51612, 95)

In [29]:
# Data Pre-processing
# Scale the training and testing data using the `StandardScaler` from `sklearn`

# Create the StandardScaler instance
scaler = StandardScaler()


In [30]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
X_scaler

StandardScaler()

In [31]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
# Ensemble Learners

# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [33]:
# Calculated the balanced accuracy score
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [34]:
# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.995931415286254

In [35]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,28,59
Actual 1,11,17107


In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.72      0.32      1.00      0.44      0.57      0.30        87
   low_risk       1.00      1.00      0.32      1.00      0.57      0.34     17118

avg / total       1.00      1.00      0.33      1.00      0.57      0.34     17205



In [37]:
# List the features sorted in descending order by feature importance
feature_imp = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
feature_imp = feature_imp.rename(columns={0:'importance', 1:'feature'}).set_index('feature')

# show top ten features
feature_imp.iloc[:10]

,importance
feature,
total_rec_prncp,0.078120
total_rec_int,0.075961
total_pymnt_inv,0.066417
total_pymnt,0.063986
last_pymnt_amnt,0.060087
out_prncp_inv,0.021583
out_prncp,0.020838
installment,0.020227
dti,0.016717


In [38]:
# Easy Ensemble Classifier

# Train the Classifier
ensemble = EasyEnsembleClassifier(n_estimators = 500,random_state=1)
ensemble = ensemble.fit(X_train_scaled, y_train)

In [39]:
# Calculated the balanced accuracy score
Predictions_ensemble = ensemble.predict(X_test_scaled)
acc_score_ensemble = accuracy_score(y_test, Predictions_ensemble)
acc_score_ensemble

0.9444928799767509

In [40]:
# Display the confusion matrix
cm = confusion_matrix(y_test, Predictions_ensemble)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,79,8
Actual 1,947,16171


In [41]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, Predictions_ensemble))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



In [43]:
### Final Questions

#1. Which model had the best balanced accuracy score?

   #Balanced Random Forest Classifier had the best balanced accuracy score of 0.99 compared to easy ensemble accuracy score of 0.94

#2. Which model had the best recall score?

   # Balanced Random Forest Classifier had the best recall score of 1 compared to easy ensemble recall score of 0.94

#3. Which model had the best geometric mean score?

   # Easy ensemble had the best geometric mean score of 0.93 compared to Balanced Forest classifier geometric mean score of 0.57

#4. What are the top three features?

#total_rec_prncp = 0.078120
#total_rec_int = 0.075961
#total_pymnt_inv = 0.066417